In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-5cutcntw
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-5cutcntw
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=7835405e28f8bce248575ac00cf0ae37885e2179bb17e3b51f7299d332d9a903
  Stored in directory: /tmp/pip-ephem-wheel-cache-2e_v3201/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [1]:
  %%writefile bu.cu
 #include <iostream>
 #include <vector>
 #include <chrono>
 using namespace std;
 __device__ void device_swap(int& a, int& b) {
 int temp = a;
 a = b;
 b = temp;
 }
 __global__ void kernel_bubble_sort_odd_even(int* arr, int size) {
 bool isSorted = false;
 while (!isSorted) {
 isSorted = true;
 int tid = blockIdx.x * blockDim.x + threadIdx.x;
 if (tid % 2 == 0 && tid < size - 1) {
 if (arr[tid] > arr[tid + 1]) {
 device_swap(arr[tid], arr[tid + 1]);
 isSorted = false;
 }
 }
 __syncthreads(); // Synchronize threads within block
 if (tid % 2 != 0 && tid < size - 1) {
 if (arr[tid] > arr[tid + 1]) {
 device_swap(arr[tid], arr[tid + 1]);
 isSorted = false;
 }
 }
 __syncthreads(); // Synchronize threads within block
 }
 }
 void bubble_sort_odd_even(vector<int>& arr) {
 int size = arr.size();
 int* d_arr;
 cudaMalloc(&d_arr, size * sizeof(int));
 cudaMemcpy(d_arr, arr.data(), size * sizeof(int), cudaMemcpyHostToDevice);

 cudaMemcpy(d_arr, arr.data(), size * sizeof(int), cudaMemcpyHostToDevice);
 // Calculate grid and block dimensions
 int blockSize = 256;
 int gridSize = (size + blockSize - 1) / blockSize;
 // Perform bubble sort on GPU
 kernel_bubble_sort_odd_even<<<gridSize, blockSize>>>(d_arr, size);
 // Copy sorted array back to host
 cudaMemcpy(arr.data(), d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);
 cudaFree(d_arr);
 }
 int main() {
 vector<int> arr = {5, 2, 9, 1, 7, 6, 8, 3, 4};
 double start, end;
 // Measure performance of parallel bubble sort using odd-even transposition
 start = chrono::duration_cast<chrono::milliseconds>(chrono::system_clock::now().time_since_ep
 bubble_sort_odd_even(arr);
 end = chrono::duration_cast<chrono::milliseconds>(chrono::system_clock::now().time_since_epoc
 cout << "Parallel bubble sort using odd-even transposition time: " << end - start
return 0;
 }

Writing sum.cu


In [2]:
!nvcc bu.cu -o bu
!./bu


Minimum value: 2147483647
Maximum value: -2147483648
Sum: 0
Average: 0
